<a href="https://colab.research.google.com/github/MuhammadHassanRaza63/Fruit-Vegi_Classification/blob/main/DeepFakeProjectipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kagglehub timm torchmetrics pytorch-lightning grad-cam opencv-python

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.0/983.0 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 828.2/828.2 kB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 M

In [2]:
#download dataset
import kagglehub, os

path = kagglehub.dataset_download("xhlulu/140k-real-and-fake-faces")
print("Dataset path:", path)

Dataset path: /kaggle/input/140k-real-and-fake-faces


In [3]:
#Correct paths based on your structure
train_dir = os.path.join(path, "real_vs_fake", "real-vs-fake", "train")
val_dir   = os.path.join(path, "real_vs_fake", "real-vs-fake", "valid")
test_dir  = os.path.join(path, "real_vs_fake", "real-vs-fake", "test")

print("Train folder:", train_dir)
print("Valid folder:", val_dir)
print("Test folder:", test_dir)

Train folder: /kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/train
Valid folder: /kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/valid
Test folder: /kaggle/input/140k-real-and-fake-faces/real_vs_fake/real-vs-fake/test


In [4]:
#import lib.
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
import numpy as np
import cv2
import pytorch_lightning as pl
from torchmetrics import Accuracy, F1Score

In [5]:
#frequency domain transform
class AddFFT(object):
    """Adds FFT magnitude channel to image tensor."""
    def __call__(self, img):
        img_np = np.array(img)
        fft = np.fft.fft2(cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY))
        fft_shift = np.fft.fftshift(fft)
        magnitude_spectrum = 20 * np.log(np.abs(fft_shift) + 1)
        magnitude_spectrum = cv2.normalize(magnitude_spectrum, None, 0, 255, cv2.NORM_MINMAX)
        magnitude_spectrum = np.uint8(magnitude_spectrum)
        magnitude_spectrum = np.expand_dims(magnitude_spectrum, axis=2)
        img_np = np.concatenate([img_np, magnitude_spectrum], axis=2)
        return torch.tensor(img_np).permute(2, 0, 1).float() / 255.0

In [6]:
#DataLoaders
tfm_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    AddFFT()
])

tfm_val = transforms.Compose([
    transforms.Resize((224,224)),
    AddFFT()
])

train_ds = datasets.ImageFolder(train_dir, transform=tfm_train)
val_ds   = datasets.ImageFolder(val_dir, transform=tfm_val)
test_ds  = datasets.ImageFolder(test_dir, transform=tfm_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=2)

In [7]:
#model Defination
class DeepfakeDetector(pl.LightningModule):
    def __init__(self, lr=1e-4):
        super().__init__()
        self.backbone = timm.create_model("xception41", pretrained=True, in_chans=4, num_classes=2)
        self.criterion = nn.CrossEntropyLoss()
        self.acc = Accuracy(task="multiclass", num_classes=2)
        self.f1 = F1Score(task="multiclass", num_classes=2)
        self.lr = lr

    def forward(self, x):
        return self.backbone(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", self.acc(preds, y), prog_bar=True)
        self.log("train_f1", self.f1(preds, y), prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        preds = torch.argmax(logits, dim=1)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.acc(preds, y), prog_bar=True)
        self.log("val_f1", self.f1(preds, y), prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr)

In [ ]:
#Training
model = DeepfakeDetector(lr=1e-4)
trainer = pl.Trainer(max_epochs=5, accelerator="auto", devices=1)
trainer.fit(model, train_loader, val_loader)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/108M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | backbone  | XceptionAligned    | 24.9 M | train
1 | criterion | CrossEntropyLoss   | 0      | train
2 | acc       | MulticlassAccuracy | 0      | train
3 | f1        | MulticlassF1Score  | 0      | train
---------------------------------------------------------


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]